In [1]:
from light_curves import LightCurve
from exposures import Calexp
from task import Run
import numpy as np

In [ ]:
n_points = [10,30,50,70,100]
levels = [20,11,10,9]


for n in n_points:
    for lvl in levels:
        process = Run(name = f"run_{n}pts_{lvl}lvl", htm_level=lvl)
        process.inject_task()
        schema = process.measure_task()
        params = [{"t_0": 60700.5, "t_E": 110, "u_0": 0.5, "m_base": m} for m in np.linspace(18,24,n)]
        process.add_lc(params[0], ra=57.59451632893858, dec=-32.481152201226145)
        for p in params[1:]:
            process.add_lc(p, dist=0.2)
        process.log_task("Add and simulate light curves")
        for j, dataId in enumerate(process.calexp_dataIds[:10]):
            print(f" ------ CALEXP {j} ------")
            calexp = Calexp(dataId)
            injected_exp, injected_points = process.inject_calexp(calexp, save_fit=f"calexp_{j}.fit")
            if injected_exp != None:
                sources = process.measure_calexp(injected_exp, schema)
                flux, flux_err = process.find_flux(sources, injected_points, save=dataId)
                for f, ferr, lc in zip(flux, flux_err, process.inj_lc):
                    mag, mag_err = calexp.get_mag(f, ferr)
                    lc.add_mag(f, ferr, dataId)
                    print(f"ra = {lc.ra}, dec = {lc.dec}")
                    print("Measured ", mag, mag_err)
                    print("Injected ", lc.data["mag_sim"][j])
        process.save_lc()
        process.save_time_log()
        process.time_analysis()

(ra,dec) = (57.59451632893858, -32.481152201226145)/nHTM_ID = 9871854450032 - HTM_level=20 (bounded by a circle of radius ~0.27 arcsec.)


/opt/lsst/software/stack/conda/envs/lsst-scipipe-9.0.0/share/eups/Linux64/daf_butler/g588e17ab67+fac83bd63d/python/lsst/daf/butler/registry/queries/_structs.py:401: FutureWarning: Dimension 'htm20' should no longer be used in data IDs. Use the region 'OVERLAPS' operator in the where clause instead. Will be removed after v28.
  warnings.warn(


Found 145 calexps


/home/karennowo/DP0_microlensing/light_curves.py:107: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.data = pd.concat([self.data, new_data], ignore_index=True)


 ------ CALEXP 0 ------
Points injected: 3
[0, 3, 5]


...

Searching in lc 0
Searching in lc 3
Searching in lc 4
Searching in lc 5
Searching in lc 7
Searching in lc 8
Searching in lc 9
Searching in lc 12
Searching in lc 13
Searching in lc 15
Searching in lc 20
Searching in lc 24
Searching in lc 25
Searching in lc 28
ra = 57.59451632893858, dec = -32.481152201226145
Measured  22.032655432565285 0.043783976984466615
Injected  17.999526457030296
ra = 57.721693928416556, dec = -32.61193236702722
Measured  22.032655432565285 0.043783976984466615
Injected  18.206423008754435
ra = 57.68061138727324, dec = -32.30962597321138
Measured  22.032655432565285 0.043783976984466615
Injected  18.413319560478573
ra = 57.50916802459406, dec = -32.48486065570155
Measured  22.032655432565285 0.043783976984466615
Injected  18.62021611220271
ra = 57.50395607562979, dec = -32.64262458704941
Measured  22.032655432565285 0.043783976984466615
Injected  18.827112663926847
ra = 57.488692246461895, dec = -32.5547566553623
Measured  22.032655432565285 0.043783976984466615
I

Searching in lc 1
Searching in lc 16
Searching in lc 17
Searching in lc 18
Searching in lc 22
Searching in lc 25
Searching in lc 26
Searching in lc 29
ra = 57.59451632893858, dec = -32.481152201226145
Measured  23.630383816643523 0.1814315053372724
Injected  17.999526450553148
ra = 57.721693928416556, dec = -32.61193236702722
Measured  23.630383816643523 0.1814315053372724
Injected  18.206423002277287
ra = 57.68061138727324, dec = -32.30962597321138
Measured  23.630383816643523 0.1814315053372724
Injected  18.413319554001426
ra = 57.50916802459406, dec = -32.48486065570155
Measured  23.630383816643523 0.1814315053372724
Injected  18.62021610572556
ra = 57.50395607562979, dec = -32.64262458704941
Measured  23.630383816643523 0.1814315053372724
Injected  18.8271126574497
ra = 57.488692246461895, dec = -32.5547566553623
Measured  23.630383816643523 0.1814315053372724
Injected  19.03400920917384
ra = 57.63692147316318, dec = -32.66239828628203
Measured  23.630383816643523 0.181431505337272

Searching in lc 0
Searching in lc 4
Searching in lc 5
Searching in lc 6
Searching in lc 7
Searching in lc 8
Searching in lc 15
Searching in lc 20
Searching in lc 22
Searching in lc 24
Searching in lc 28
ra = 57.59451632893858, dec = -32.481152201226145
Measured  23.66222447815353 0.16068322335122615
Injected  17.999520247493123
ra = 57.721693928416556, dec = -32.61193236702722
Measured  23.66222447815353 0.16068322335122615
Injected  18.206416799217262
ra = 57.68061138727324, dec = -32.30962597321138
Measured  23.66222447815353 0.16068322335122615
Injected  18.4133133509414
ra = 57.50916802459406, dec = -32.48486065570155
Measured  23.66222447815353 0.16068322335122615
Injected  18.620209902665536
ra = 57.50395607562979, dec = -32.64262458704941
Measured  23.66222447815353 0.16068322335122615
Injected  18.827106454389675
ra = 57.488692246461895, dec = -32.5547566553623
Measured  23.66222447815353 0.16068322335122615
Injected  19.034003006113814
ra = 57.63692147316318, dec = -32.6623982

Searching in lc 3
Searching in lc 9
Searching in lc 12
Searching in lc 13
Searching in lc 23
ra = 57.59451632893858, dec = -32.481152201226145
Measured  24.195586373736916 0.2624112728228713
Injected  17.999520247493123
ra = 57.721693928416556, dec = -32.61193236702722
Measured  24.195586373736916 0.2624112728228713
Injected  18.206416799217262
ra = 57.68061138727324, dec = -32.30962597321138
Measured  24.195586373736916 0.2624112728228713
Injected  18.4133133509414
ra = 57.50916802459406, dec = -32.48486065570155
Measured  24.195586373736916 0.2624112728228713
Injected  18.620209902665536
ra = 57.50395607562979, dec = -32.64262458704941
Measured  24.195586373736916 0.2624112728228713
Injected  18.827106454389675
 ------ CALEXP 6 ------
Points injected: 11
[0, 3, 5, 7, 8, 9, 12, 13, 20, 25, 28]


Searching in lc 0
Searching in lc 3
Searching in lc 5
Searching in lc 7
Searching in lc 8
Searching in lc 9
Searching in lc 12
Searching in lc 13
Searching in lc 20
Searching in lc 25
Searching in lc 28
ra = 57.59451632893858, dec = -32.481152201226145
Measured  23.520986148518265 0.13692369796301931
Injected  17.999520246553935
ra = 57.721693928416556, dec = -32.61193236702722
Measured  23.520986148518265 0.13692369796301931
Injected  18.206416798278074
ra = 57.68061138727324, dec = -32.30962597321138
Measured  23.520986148518265 0.13692369796301931
Injected  18.413313350002213
ra = 57.50916802459406, dec = -32.48486065570155
Measured  23.520986148518265 0.13692369796301931
Injected  18.620209901726348
ra = 57.50395607562979, dec = -32.64262458704941
Measured  23.520986148518265 0.13692369796301931
Injected  18.827106453450487
ra = 57.488692246461895, dec = -32.5547566553623
Measured  23.520986148518265 0.13692369796301931
Injected  19.034003005174625
ra = 57.63692147316318, dec = -32

In [11]:
lc = process.inj_lc[6]
ra= lc.ra*np.pi/180; dec =  lc.dec*np.pi/180
ra,dec

(1.0026102138055069, -0.5674241482716905)

In [33]:
sources[i_ra]["coord_ra"],sources[i_dec]["coord_ra"]

(Angle(57.445333125118566, degrees), Angle(57.445335357060969, degrees))

In [48]:
s = sources
i_ra = np.argmin(np.abs(s["coord_ra"] - ra))
point = s[i_dec]
# coord_ra: 1.00261 rad
# coord_dec: -0.566496 rad

In [51]:
from astropy.coordinates import SkyCoord
import numpy as np

target_coord = SkyCoord(ra=ra, dec=dec, unit="deg")
source_coords = SkyCoord(ra=s["coord_ra"], dec=s["coord_dec"], unit="deg")
distances = source_coords.separation(target_coord)
closest_index = np.argmin(distances)
point = s[closest_index]


print(f"El punto más cercano a ({ra}, {dec}) tiene coordenadas:")
print(f"RA: {closest_ra}, DEC: {closest_dec}")
print(f"Fila completa: {closest_row}")


El punto más cercano a (1.0026102138055069, -0.5674241482716905) tiene coordenadas:
RA: 1.0003741766902436, DEC: -0.5667706642386247
Fila completa:  id       coord_ra           coord_dec      parent coord_raErr coord_decErr base_SdssCentroid_x slot_Centroid_x base_SdssCentroid_y slot_Centroid_y base_SdssCentroid_xErr slot_Centroid_xErr base_SdssCentroid_yErr slot_Centroid_yErr base_SdssCentroid_flag base_CircularApertureFlux_flag_badCentroid base_GaussianFlux_flag_badCentroid base_LocalBackground_flag_badCentroid base_PsfFlux_flag_badCentroid base_SdssShape_flag_badCentroid base_Variance_flag_badCentroid slot_Centroid_flag base_SdssCentroid_flag_edge base_CircularApertureFlux_flag_badCentroid_edge base_GaussianFlux_flag_badCentroid_edge base_LocalBackground_flag_badCentroid_edge base_PsfFlux_flag_badCentroid_edge base_SdssShape_flag_badCentroid_edge base_Variance_flag_badCentroid_edge slot_Centroid_flag_edge base_SdssCentroid_flag_noSecondDerivative base_CircularApertureFlux_flag_badCe

In [52]:
flux = point["base_PsfFlux_instFlux"]; flux_err = point["base_PsfFlux_instFluxErr"]
photoCalib = calexp.expF.getPhotoCalib()
measure = photoCalib.instFluxToMagnitude(flux, flux_err)
measure.value, measure.error

(21.97602885934537, 0.0363816896447421)

In [50]:
lc.data[lc.data["detector"] == calexp.calexp_data["detector"]]
lc.data[lc.data["visit"] == calexp.calexp_data["visit"]]

,detector,visit,mjd,mag_sim,flux,flux_err,mag,mag_err
3,185,177423,59813.364525,21.99952,NaN,NaN,NaN,NaN


In [29]:
differences = np.abs(s["coord_ra"] - ra)
min_difference = np.min(differences)
np.where(differences == min_difference)


(array([2975]),)

In [18]:
i_dec = np.argmin(np.abs(s["coord_dec"] - dec))
s[i_dec]

<class 'lsst.afw.table.SourceRecord'>
id: 2189
coord_ra: 1.00261 rad
coord_dec: -0.567424 rad
parent: 0
coord_raErr: 7.59855e-08
coord_decErr: 8.26985e-08
base_SdssCentroid_x: 2637.31
base_SdssCentroid_y: 2448.52
base_SdssCentroid_xErr: 0.0959159
base_SdssCentroid_yErr: 0.0822788
base_SdssCentroid_flag: 0
base_SdssCentroid_flag_edge: 0
base_SdssCentroid_flag_noSecondDerivative: 0
base_SdssCentroid_flag_almostNoSecondDerivative: 0
base_SdssCentroid_flag_notAtMaximum: 0
base_SdssCentroid_flag_near_edge: 0
base_SdssCentroid_flag_resetToPeak: 0
base_SdssCentroid_flag_badError: 0
base_Blendedness_old: 0
base_Blendedness_raw: 0
base_Blendedness_raw_child_instFlux: 8232.28
base_Blendedness_raw_parent_instFlux: 8232.28
base_Blendedness_abs: 0
base_Blendedness_abs_child_instFlux: 8263.58
base_Blendedness_abs_parent_instFlux: 8263.58
base_Blendedness_raw_child_xx: 2.72413
base_Blendedness_raw_child_yy: 1.93772
base_Blendedness_raw_child_xy: 0.0426744
base_Blendedness_raw_parent_xx: 2.72413
base_